In [71]:
import os
import pandas as pd
import glob
import string
import nltk
import re


In [72]:
import os
import pandas as pd
import glob
import re


class Utils():
    root_directory = os.chdir('c:/Users/iZiPC.by/notebooks/parser/')# input path
    list_of_files = glob.glob('**/*.txt', recursive=True)
    invalid_parsing = []
    trash = ['Волковыск-1.txt','Волковыск.txt','Вороново-1.txt',
         'Вороново.txt', 'Черновик-1.txt', 'Черновик.txt']
    for file in list_of_files:
        try:
             with open(file, encoding='utf-8') as f:
                    num = f.readline()
                    file_ = f.read()
                    if num and file_:
                        pass
        except:
            invalid_parsing.append(file)
    for i in invalid_parsing:
        list_of_files.remove(i)
    for i in trash: 
        list_of_files.remove(i)


list_of_files = Utils.list_of_files


plt_ = []
wbc_ = []
hgb_ = []
crea_ = []
ldh_ = []
sex_ = []
crp_ = []
tr = []
bir = []
ad = []
dis = []
ID_ = []
dgs_ = []
def ID(num):
  """returns ID of the patient"""
  for i in num.split():
    for char in i.split():
        if char.isnumeric():
            try:
                return int(char)
            except:
                return 'None'
            
            
def treatment(file_):
  """ returns srting of remedies used for the patient """
  for line in file_.split('\n'):
    if re.search('лечение', line):  # Does the same thing as "if 'hello' in line:"
        return(line.replace('Проведенное лечение:', \
                            '').replace('ЛФК', '').replace('ФТЛ', ''))
    

def born_adm_disch(file_):
    """ finds all nessesary dates in the epicrisis"""
    pattern = re.compile("(\d{2}).(\d{2}).(\d{4})") # check all dates
    birthday, admission = pattern.findall(file_)[:2]
    discharging = pattern.findall(file_)[-1]
    try:
        return ':'.join(birthday), ':'.join(admission), ':'.join(discharging)
    except:
        return 'None'


pattern_diagnosis = ['Диагноз:','Диагноз клинический:','Диагноз', 'Д-з:',
                     "Диагноз заключительный:"]
def diagnosis_dirty(file_):
  int_ = 0
  for i in file_.split():
    int_ += 1
    if i in pattern_diagnosis:
      return file_.split()[int_+1:int_+40]    
  

def crp(file_):
  """returns all crp values"""
  pattern = re.sub(r'[ЦСC]РБ', 'С-реактивныйбелок', file_)
  pattern_1 = re.compile(r'(?:\w\Dреактивныйбелок|\w\Dреактивныйбелокдо)\
                         (\d*.\d+|\d+)')
  pattern_2 = pattern_1.findall(''.join(pattern.split()))
  try:
    return pattern_2
  except:
    return 'None'


def gender(file_):
  patt1 = re.compile(r'(?<=Ф.И.О:)[А-Я]\w{,20}[А-Я]\w{,20}(вич)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return 'male'
    else:
      return 'female'
  except:
      pass
  

def ldh(file_):
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d\d\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wактатдегидрогеназ\w', 'лдг', file_)
  patt1 = re.compile(r'(?<=лдг)\d{,4}')
  patt2 = patt1.findall(''.join(file_.split()))
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass



def cre(file_):
  """Returns all creatinine values"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d\d\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'(\wреатини\w|креатин|креат)(?![а-яА-Я,])', 'cre', file_)
  patt1 = re.compile(r'(?<=cre)\d{,3}')
  patt2 = patt1.findall(''.join(file_.split()))
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass


def hgb(file_):
  """returns list of str with level of hemoglobin"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  patt = re.compile(r'гемоглобин|гемогл|гб|гем')
  patt1 = re.sub(patt, r'hgb', file_)
  patt2 = re.compile(r'(?<=hgb)\d{2,3}')
  all_hgb = patt2.findall(''.join(patt1.split()))
  try:
    if all_hgb:
      return all_hgb
    else:
      return 'None'
  except:
    pass


def wbc(file_):
  """returns list of str with level of wbc"""
  file_ = file_.lower()
  patt1 = re.sub(r'[():]', '', file_)
  patt1 = re.sub(r'10(\*|[еe])9', '', patt1)
  patt1 = re.sub(r'\Dбщийанализкрови', 'оак', patt1) 
  patt2 = re.sub(r'(?<=оак)\d\d.\d\d.\d{2,4}|(?<=оак)\d\d.\d\d', '', patt1)
  patt3 = re.compile(r'(?<=оак|wbc)(?:л|лейкоцит\w)(\d*.\d+|\d+)')
  all_wbc = patt3.findall(''.join(patt2.split()))
  try:
    if all_wbc:
      return all_wbc
    else:
      return 'None'
  except:
    pass


def plt(file_):
  """returns list of str with level of plt"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'10(\*|[еe])9', '', file_)
  patt2 = re.compile(r'(?:\wромбоцит\w|тр)')
  patt1 = re.sub(patt2, '', file_)
  patt1 = ''.join(patt1.split())
  patt2 = re.compile(r'(?<=plt)\d{3}')
  all_plt = patt2.findall(patt1)
  try:
    if all_plt:
      return all_plt
    else:
      return 'None'
  except:
    pass

In [73]:
for file in list_of_files:
    try:
         with open(file, encoding='utf-8') as f:
                num = f.readline()
                file_ = f.read()
                ID_.append(ID((num)))
                tr.append(treatment((file_)))
                bir.append(born_adm_disch(file_)[0])
                ad.append(born_adm_disch(file_)[1])
                dis.append(born_adm_disch(file_)[2])
                dgs_.append(diagnosis_dirty((file_)))
                crp_.append(crp(''.join(file_.split())))
                sex_.append(gender(''.join(file_.split())))
                ldh_.append(ldh(file_))
                crea_.append(cre(file_))
                hgb_.append(hgb(file_))
                wbc_.append(wbc(file_))
                plt_.append(plt(file_))
    except:
        print(file)

In [74]:
len(dgs_)

1998

In [75]:
diagnosis_cleaned =  []
for diagnos in dgs_:
    try:
        diagnosis_cleaned.append(' '.join(diagnos))
    except:
        diagnosis_cleaned.append('None')

In [76]:
len(diagnosis_cleaned)

1998

In [77]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [111]:
list_wo_punct = []
for sent in diagnosis_cleaned:
    try:
        sent = remove_punctuation(sent.lower())
        list_wo_punct.append(sent)
    except:
        pass

In [112]:
len(list_wo_punct)

1998

In [140]:
# def coid19_finder(file_):
#     covid_patterns = ['covid19', 'коронавирусная', "короновирусная", "корон", "covid19", 'торсков2', 'sarscov2',"b342" ]
#     covid_list = []
#     for patt in covid_patterns:
#         for sent in file_:
#             pattern = re.sub(r'patt', 'covid', sent)
#             pattern_1 = re.compile(r'covid')
#             patt_2 = pattern_1.findall(''.join(pattern.split()))
#             try:
#                 covid_list.append(patt_2)
#             except:
#                 pass
#     c =[]
#     for i in covid_list:
#         if not i:
#             c.append(0)
#         else:
#             c.append(1)
#     return c

In [143]:
#k = coid19_finder(list_wo_punct)

In [157]:
covid_list = []

In [ ]:
c =[]
for i in covid_list:
    if not i:
        c.append(0)
    else:
        c.append(1)

In [158]:
for sent in list_wo_punct:
    pattern = re.sub(r'covid19', 'covid', sent )
    pattern = re.sub(r'коронавирусная', 'covid', pattern)
    pattern = re.sub(r'короновирусная', 'covid', pattern)
    pattern = re.sub(r'b342', 'covid', pattern)
    pattern = re.sub(r'sarscov2', 'covid', pattern)
    pattern = re.sub(r'торсков2', 'covid', pattern)
    pattern_1 = re.compile(r'covid')
    patt_2 = re.search(pattern_1, pattern)
    #patt_2 = pattern_1.findall(''.join(pattern.split()))
    try:
        covid_list.append(patt_2)
    except:
        pass

In [159]:
c =[]
for i in covid_list:
    if not i:
        c.append(0)
    else:
        c.append(1)

In [168]:
dfc={}
for i in c:
    dfc[i] = dfc.get(i, 0)+1
dfc

{0: 1783, 1: 215}

In [167]:
len(c)

1998

In [122]:
list_wo_punct[19]

'поступлении внегоспитальная пневмония в нижней доле левого легкого дн 1 ст диагноз заключительный инфекция covid19 пцр от 03052020 реконвалесцент внегоспитальная пневмония в средней доле правого легкого верхней доле s3s4s5 и нижней s910 левого легкого средней степени тяжести с исходом'

In [163]:
data_dct = {'Platlets':plt_, 'Leucocytes': wbc_, 'Hemoglobin': hgb_,
           'Creatinine': crea_, 'LDG':ldh_, 'Gender':sex_, 'CRP':crp_,
                                  'Treatment':tr, 'Birthday': bir, 'Admission':ad,
                                  'Discharge':dis, 'ID':ID_, 'Diagnosis': list_wo_punct, 'COVID-19': c }

In [164]:
dataFrame = pd.DataFrame(data_dct)


In [165]:
dataFrame.head(20)

,Platlets,Leucocytes,Hemoglobin,Creatinine,LDG,Gender,CRP,Treatment,Birthday,Admission,Discharge,ID,Diagnosis,COVID-19
0,[320],[5.81],[151],None,None,male,[],"левофлоксацин . амброксол , парацетамол , иб...",12:09:1988,23:09:2019,02:10:2019,5616.0,внегоспитальная полисегментарная пневмония в н...,0
1,[444],[7.15],[138],[86],None,female,[],"триксоцеф. азитромицин, амброксол,",22:10:1998,18:01:2018,29:01:2018,430.0,j18 пневмония без уточнения возбудителя основн...,0
2,"[148, 371, 159]","[8.2, 5.76, 7.79]","[146, 164, 165]",None,None,female,[],"цефтриаксон, амброксол, плевилокс,",10:05:1963,17:02:2020,13:03:2020,1103.0,j18 пневмония без уточнения возбудителя основн...,0
3,None,None,None,None,None,female,[],None,03:07:1940,18:05:2018,01:06:2018,3338.0,none,0
4,[236],[10.42],[146],None,None,male,[],"цефтриаксон . амброкосл . аспикард , аторв...",07:09:1957,22:01:2019,30:01:2019,498.0,внегоспитальная пневмония в нижней доле справа...,0
5,"[225, 184, 239]","[6.57, 8.42, 6.05]","[133, 158, 138]",[81],None,female,[],"бикард, лазартан, моксинидин, триксоцеф, амбро...",20:05:1941,07:06:2017,15:06:2017,3846.0,внегоспитальная пневмония в нижней доле правог...,0
6,None,None,[141],None,None,male,[],"цефтриаксон , амброксол , КПС , тамсулозин , а...",10:05:1939,16:02:2020,19:02:2020,1065.0,туберкулез костей стриктура мочеточника н3 ту...,0
7,[198],[8.24],[132],None,None,female,[],"левофлоксацин . амброксол . метопролол , аспик...",08:01:1954,21:12:2019,27:12:2019,7656.0,none,0
8,"[344, 189, 414, 347]","[10.54, 10.57, 9.8, 8.66]","[138, 139, 132, 132]",None,None,female,[],"цефасульбактам,амброксол,аспикард, индап, лиз...",02:10:1949,29:12:2018,21:01:2019,8182.0,внегоспитальная полисегментарная в ндоле прав...,0
9,"[189, 206, 206, 203]","[7.82, 10.34, 10.34, 8.82]","[147, 148, 148, 159]",[99],[264],male,[],"дексаметазон, цефтриаксон, ацц, гепарин, фамо...",17:10:1958,20:11:2020,01:12:2020,4979.0,b342 среднетяжёлая короновирусная sarscov2 пцр...,1


In [156]:
dataFrame[dataFrame['ID'] == 635.0]

,Platlets,Leucocytes,Hemoglobin,Creatinine,LDG,Gender,CRP,Treatment,Birthday,Admission,Discharge,ID,Diagnosis
31,"[364, 191]","[4.56, 8.27]","[123, 155]",None,None,male,"[7,7]","левофлоксацин , диклофенак , амброксол , омеп...",05:06:1981,28:01:2020,07:02:2020,635.0,"[Внегоспитальная, плевропневмония, в, н/д, спр..."


In [153]:
len(dataFrame['CRP'])

1998